In [1]:
print("Hello")

Hello


## Creating runnables to solve this problem 

In [2]:
from abc import ABC, abstractmethod

class Runnables(ABC):
    
    @abstractmethod
    def invoke(input_data):
        pass
    
    
    

In [3]:
import random, warnings

class FakeLLM(Runnables):

    def __init__(self):
        print("LLM Created")
        
    def invoke(self, prompt):
        respone_list = [
            "Kathmandu is the captial of Nepal",
            "AI Stands for Artifical Intelligence",
            "Lumbini Lions Won the 2nd NPL Trophy",
        ]
        
        return {"response": random.choice(respone_list)}
    
    
    def predict(self, prompt):
        
        warnings.warn("The predict method is going to be deprecated in the future! Please use invoke method instead.", UserWarning)
        
        respone_list = [
            "Kathmandu is the captial of Nepal",
            "AI Stands for Artifical Intelligence",
            "Lumbini Lions Won the 2nd NPL Trophy",
        ]
        
        return {"response": random.choice(respone_list)}

In [4]:
### Prompt Template

class DummyPromptTemplate:
    
    def __init__(self, template, input_variables):
        
        self.template = template
        self.input_variables = input_variables
        
    def invoke(self, input_dict):
        return self.template.format(**input_dict)
        
    def format(self, input_dict):
        warnings.warn("The format method is going to be deprecated in the future! Please use invoke method instead.", UserWarning)
        return self.template.format(**input_dict)
    
    

In [5]:
class DummyStrOutputParser(Runnables):
    
    def __init__(self):
        pass
    
    def invoke(str, input_data):
        return input_data['response']

In [6]:
class RunnableConnector(Runnables):
    
    
    def __init__(self, runnables_list):
        self.runnables_list = runnables_list
        
    def invoke(self, input_data):
       
        for runnable in self.runnables_list:
            input_data = runnable.invoke(input_data)
            # runnable.invoke(input_data)
        
        return input_data

In [7]:
template = DummyPromptTemplate(
    template="Write a {length} poem about {topic}",
    input_variables=['length','topic']
)

template.format({'length' : "short", 'topic': "Nepal"})

C:\Temp\ipykernel_23908\1417494561.py:14: UserWarning: The format method is going to be deprecated in the future! Please use invoke method instead.
  warnings.warn("The format method is going to be deprecated in the future! Please use invoke method instead.", UserWarning)


'Write a short poem about Nepal'

In [8]:
llm = FakeLLM()

LLM Created


In [9]:
chain = RunnableConnector([template, llm])

chain.invoke({'length':'medium', 'topic': 'Nepal'})

{'response': 'AI Stands for Artifical Intelligence'}

In [10]:
parser = DummyStrOutputParser()

In [11]:
chain = RunnableConnector([template, llm, parser])

chain.invoke({'length':'medium', 'topic': 'Nepal'})

'Kathmandu is the captial of Nepal'

### Creating a combined runnables from two runnables created  - multiple chains

In [15]:
template1 = DummyPromptTemplate(
    template="Write a joke about {topic}",
    input_variables= ['topic']
)

template2 = DummyPromptTemplate(
    template="Explain the following joke {response}",
    input_variables= ['response']
)

In [16]:
parser1 = DummyStrOutputParser()

chain1 = RunnableConnector([template1, llm])

# chain1.invoke({'topic': 'AI'})

In [17]:
chain2 = RunnableConnector([template2, llm, parser])

final_chain = RunnableConnector([chain1, chain2])

In [18]:
final_chain.invoke({'topic': "AI"})

'Lumbini Lions Won the 2nd NPL Trophy'